In [1]:
# preprocess_LSTM_dask.py
# -----------------------------------------------------------
# ❶  Imports & Dask cluster
# -----------------------------------------------------------
import torch, zarr
import geopandas as gpd
from pathlib import Path
from collections import Counter, defaultdict
from dask import delayed, compute
from dask.distributed import LocalCluster, Client, performance_report
from IPython.display import display # Dask dashboard inside the notebook itself:
import traceback, sys, dask, pickle
import pandas as pd, numpy as np, xarray as xr
import os, multiprocessing, torch, psutil, os, tempfile, json, shutil, time
import urllib.error
from pathlib import Path
from dask.distributed import Client, LocalCluster
from functools import lru_cache
from collections import defaultdict
from dask.distributed import Client, LocalCluster, wait, as_completed

from data_access import era5, hres
from utils import build_sample, samples_to_xarray, load_streamflow_for_gauge,  build_sample_from_disk, get_usgs_streamflow, get_or_download_streamflows
# from config import FORECAST_BLOCKS, SCRATCH

NameError: name 'pd' is not defined

In [ ]:

print("Logical CPU cores :", multiprocessing.cpu_count())

if torch.cuda.is_available():
    print("CUDA device      :", torch.cuda.get_device_name(0))
    print("GPU capability   :", torch.cuda.get_device_capability(0))
else:
    print("No CUDA‑capable GPU detected")

print("RAM (GB total)   :", round(psutil.virtual_memory().total / 1e9, 1))

In [ ]:


# ------------------------------------------------------------------
# paths & constants -------------------------------------------------
BASE_OBS  = Path('/Projects/HydroMet/currierw/ERA5_LAND')
BASE_FCST = Path('/Projects/HydroMet/currierw/HRES')
SCRATCH   = Path('/Projects/HydroMet/currierw/HRES_processed_tmp')
FINAL_OUT = Path('/Projects/HydroMet/currierw/HRES_processed')

FORECAST_BLOCKS = {
    "train":      pd.date_range('2016-01-01', '2020-09-30', freq='5D'),
    "validation": pd.date_range('2020-10-01', '2022-09-30', freq='5D'),
    "test":       pd.date_range('2022-10-01', '2024-09-30', freq='5D'),
}
EXPECTED_LEN = 106


# ---------------------------- Main driver -----------------------------
if __name__ == "__main__":
    BATCH_SIZE = 2_000
    SAVE_INTERVAL = 2_000

    # ---- Clear scratch output folder ----
    if SCRATCH.exists():
        shutil.rmtree(SCRATCH)
    SCRATCH.mkdir(parents=True, exist_ok=True)

    # ---- Load gauges ----
    gdf = gpd.read_file('/Projects/HydroMet/currierw/Caravan-Jan25-csv/shapefiles/camels/camels_basin_shapes.shp')
    gauge_ids = gdf["gauge_id"].str.split("_").str[-1].tolist()
    streamflows, skipped = get_or_download_streamflows(gdf)

    print(f"Loaded {len(streamflows)} gauges. Skipped {len(skipped)}.")

    # ---- Dask cluster setup ----
    module_path = '/home/wcurrier/LSTM_runoff_tests'

    def add_module_path(dask_worker):
        if module_path not in sys.path:
            sys.path.insert(0, module_path)

    cluster = LocalCluster(n_workers=16, threads_per_worker=2, memory_limit="8GB")
    cluster.adapt(minimum=8, maximum=32)
    client = Client(cluster)
    client.run(add_module_path)

    # ---- Task submission in batches ----
    all_futures = []
    pending = []

    for gauge_id in streamflow[:10]s:
        for split, dates in FORECAST_BLOCKS.items():
            for d in dates[:5]:
                future = client.submit(build_sample_from_disk, gauge_id, split, d, pure=False)
                pending.append(future)

                if len(pending) >= BATCH_SIZE:
                    print(f"Submitting batch of {len(pending)} tasks...")
                    wait(pending, timeout="10m")
                    all_futures.extend(pending)
                    pending = []

    # Final pending batch
    if pending:
        print(f"Submitting final batch of {len(pending)} tasks...")
        wait(pending, timeout="10m")
        all_futures.extend(pending)

    # ---- Collect results as they complete ----
    print("Waiting for results and saving intermediate files...")
    start_time = time.time()
    grouped = defaultdict(list)
    counter = 0

    for future, result in as_completed(all_futures, with_results=True):
        if result is not None:
            grouped[(result["basin_id"], result["split"])].append(result)
            counter += 1

        if counter % SAVE_INTERVAL == 0 and counter > 0:
            print(f"[{counter}] Saving intermediate results at {time.time() - start_time:.1f}s")
            for (gauge, split), samples in grouped.items():
                ds = samples_to_xarray(samples)
                out = SCRATCH / f"{split}_{gauge}_{counter}.nc"

                if ds.dynamic_inputs.isnull().any() or ds.targets.isnull().any():
                    print(f"[{split}][{gauge}] contains NaNs — skipping {out.name}")
                    continue

                ds.to_netcdf(out)
                print("Wrote", out)

            grouped.clear()

    # ---- Final save ----
    print("Final save of remaining results...")
    summary = []

    for (gauge, split), samples in grouped.items():
        ds = samples_to_xarray(samples)
        out = SCRATCH / f"{split}_{gauge}.nc"

        if ds.dynamic_inputs.isnull().any() or ds.targets.isnull().any():
            print(f"[{split}][{gauge}] contains NaNs — skipping final save")
            continue

        ds.to_netcdf(out)
        print("Wrote", out)

        summary.append({
            "gauge_id": gauge,
            "split": split,
            "n_samples": len(samples),
            "filename": out.name
        })

    # ---- Summary output ----
    pd.DataFrame(summary).to_csv(SCRATCH / "processed_summary.csv", index=False)
    print(f"Done. Total time: {(time.time() - start_time)/60:.1f} minutes")
    print(f"Results saved to {SCRATCH}")

# Code to concatenate indivudal files

In [ ]:
# import xarray as xr
# from pathlib import Path
# import numpy as np

# SCRATCH = Path('/Projects/HydroMet/currierw/HRES_processed_tmp')
# FINAL_OUT = Path('/Projects/HydroMet/currierw/HRES_processed')
# # Step 1: List of splits
# splits = ["train", "validation", "test"]

# for split in splits:
#     files = sorted(SCRATCH.glob(f"{split}_*.nc"))
#     valid_datasets = []

#     print(f"\n🧪 Checking {len(files)} files for '{split}' split...")

#     for f in files:
#         ds = xr.open_dataset(f)

#         # Check for NaNs
#         has_nan = (
#             np.isnan(ds["dynamic_inputs"]).any() or
#             np.isnan(ds["targets"]).any()
#         )

#         if has_nan:
#             print(f"⚠️ Skipping file with NaNs: {f.name}")
#             continue

#         valid_datasets.append(ds)

#     if not valid_datasets:
#         print(f"❌ No valid files found for split: {split}")
#         continue

#     print(f"✅ {len(valid_datasets)} valid files found for '{split}'")

#     # Step 2: Concatenate along the sample dimension
#     combined = xr.concat(valid_datasets, dim="sample")

#     # Step 3: Write out combined file
#     output_path = FINAL_OUT / f"{split}.nc"
#     combined.to_netcdf(output_path)
#     print(f"📦 Wrote combined dataset for '{split}' → {output_path}")

In [ ]:
# # Optional cleanup
# for f in SCRATCH.glob("*.nc"):
#     f.unlink()  # or f.rename(SCRATCH / "archive" / f.name)